In [0]:
from google.colab import files, auth, drive

In [0]:
# Mount to drive
drive.mount('/content/gdrive', force_remount=True)
data_dir_drive ='/content/gdrive/My Drive/Colab Notebooks/NLP1/Lab3'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [355]:
! pip install nltk

In [356]:
import nltk
import numpy as np

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [357]:
from nltk import CFG, Nonterminal
grammar1 = nltk.CFG.fromstring("""  S -> NP VP | TO VB
                                    VP -> V NP | V NP PP | V S | V PP
                                    PP -> P NP  
                                    V -> "caught" | "ate" | "likes" | "like" | "chase" | "go"
                                    NP -> Det N | Det N PP | PRP
                                    Det -> "the" | "a" | "an" | "my" | "some"
                                    N -> "mice" | "cat" | "dog" |  "school"
                                    P -> "in" | "to" | "on"
                                    TO -> "to"
                                    PRP -> "I"  """)

sent=["I", "like", "my", "dog"]
rdp = nltk.RecursiveDescentParser(grammar1)
for tree in rdp.parse(sent):
    print(tree)

(S (NP (PRP I)) (VP (V like) (NP (Det my) (N dog))))


In [0]:
nltk.app.rdparser()

In [359]:
srp = nltk.ShiftReduceParser(grammar1)
sent=["I", "like", "my", "dog"]
for tree in srp.parse(sent):
    print(tree)

(S (NP (PRP I)) (VP (V like) (NP (Det my) (N dog))))


In [0]:
nltk.app.srparser()

# **Left corner parser**
The Left-Corner Parser uses also a top-down strategy. A left-corner parser does some preprocessing before the parsing itself. It creates an association between each non-terminal label and a list of all possible left corners (start of the expression). Before applying a production from the context free grammar, it searches for the next word that there is one starting label (in the left corners list) that applies to it. (http://cs.union.edu/~striegnk/courses/nlp-with-prolog/html/node53.html?fbclid=IwAR1957LMxy1A9OTm2gpzZFzrpzo97BzRWFMKXSHOdzZoAC1NEyOdmD3hEJ4)

In [360]:
grammar2 = nltk.CFG.fromstring("""  S -> NP VP
                                    NP -> Det N | PN 
                                    VP -> IV | vincent
                                    Det -> the
                                    N -> robber
                                    PN -> vincent
                                    IV -> died  """)

string = 'vincent died'
grammar2.productions()

[S -> NP VP,
 NP -> Det N,
 NP -> PN,
 VP -> IV,
 VP -> vincent,
 Det -> the,
 N -> robber,
 PN -> vincent,
 IV -> died]

In [361]:
start = grammar2.start()
print(start)

S


In [362]:
nltk.help.upenn_tagset('S')

SYM: symbol
    % & ' '' ''. ) ). * + ,. < = > @ A[fj] U.S U.S.S.R * ** ***


In [0]:
def recognize(grammar, N):
    return grammar.productions(lhs=N)

In [0]:
def find_productions_by_left_corners(grammar, N):
    return grammar.productions(rhs=N) # start with N e.g. ... -> N...

In [0]:
def get_left_corner(production):
    return production.rhs()[0]

In [366]:
from collections import deque

def Bottom_up_Parsing(word, top, grammar):
    r = [] #result
    q = deque() 
    start_productions = find_productions_by_left_corners(grammar, word) # add production where word is left corner
    start_productions += find_productions_by_left_corners(grammar, Nonterminal(word))
    #print(start_productions)
    if len(start_productions) == 0:
        return False
    else:
        res = []
        for production in start_productions:
            q.append(production)
            if production.lhs() == top:
                res.append([production]) # i have match
        if len(res):
            return res

    while len(q) != 0: # i have productions
        curr_production = q[0] # get first production
        q.popleft() 
        #print(curr_production)

        productions = find_productions_by_left_corners(grammar, curr_production.lhs()) # add new production where current start symbol is left-corner
        #print(productions)
        #print()
        r.append((curr_production, len(productions))) # add current production in result

        while r[-1][1] == 0: #clean result list
            r.pop(-1)
            if len(r):
                r[-1] = (r[-1][0], r[-1][1]-1) 
            else:
                break

        for production in productions:
            if production.lhs() == top:
                r.append((production, None))
                f = []
                for elem in r:
                    f.append(elem[0])
                    
                res.append(f)
                r = r[:1]
            q.appendleft(production)
    
    if res == []:
        return False
    else:
        return res

Bottom_up_Parsing('am', Nonterminal('VP'), grammar3)

[[VBP -> 'am', VP -> VBP NP], [VBP -> 'am', VP -> VBP VP]]

In [367]:
grammar1.productions()

[S -> NP VP,
 S -> TO VB,
 VP -> V NP,
 VP -> V NP PP,
 VP -> V S,
 VP -> V PP,
 PP -> P NP,
 V -> 'caught',
 V -> 'ate',
 V -> 'likes',
 V -> 'like',
 V -> 'chase',
 V -> 'go',
 NP -> Det N,
 NP -> Det N PP,
 NP -> PRP,
 Det -> 'the',
 Det -> 'a',
 Det -> 'an',
 Det -> 'my',
 Det -> 'some',
 N -> 'mice',
 N -> 'cat',
 N -> 'dog',
 N -> 'school',
 P -> 'in',
 P -> 'to',
 P -> 'on',
 TO -> 'to',
 PRP -> 'I']

In [368]:
x = Bottom_up_Parsing('to', Nonterminal('S'), grammar1)
print(x)

[[TO -> 'to', S -> TO VB]]


In [369]:
x = Bottom_up_Parsing('robber', Nonterminal('N'), grammar2)
print(x)

[[N -> robber]]


In [0]:
def left_corner_parser(grammar, string):

    words = string.split()
    flag = grammar.start() #top-down
    index = 0

    def left_corner_parser_aux(grammar = grammar, index = index, flag = flag):
        #print(index, flag)
        rest = 0
        productions = Bottom_up_Parsing(words[index], flag, grammar)

        if productions == False:
            return False
        else:
            #print(productions[:][::-1])
            pass
        returnValue = True
        for productionList in productions:
            for production in productionList[1:]:
                if len(production.rhs()) == 1:
                    continue # I'm ok, there is no need for top-down
                else: # need top-down step
                    for symbol in production.rhs()[1:]:
                        index += 1
                        if index >= len(words): #case for insufficient words
                            returnValue = False
                            break
                        rest = left_corner_parser_aux(grammar, index, symbol)
                        if not rest:
                            returnValue = False
                            #print(index, symbol, production)
                            break
                        else:
                            returnValue = True
            index -= 1
            if returnValue:
                print(productionList)
                break
        #print(index, flag, returnValue)
        return returnValue

    val = left_corner_parser_aux()
    return val

In [514]:
val = left_corner_parser(grammar2, 'the robber died')
print(val)

[N -> robber]
[IV -> died, VP -> IV]
[Det -> the, NP -> Det N, S -> NP VP]
True


In [507]:
val = left_corner_parser(grammar1, 'I like my dog')
print(val)

[N -> 'dog']
[Det -> 'my', NP -> Det N]
[V -> 'like', VP -> V NP]
[PRP -> 'I', NP -> PRP, S -> NP VP]
True


In [508]:
grammar3 = nltk.CFG.fromstring("""  S -> NP VP
                                    S -> VP
                                    NP -> DT NN
                                    NP -> DT JJ NN
                                    NP -> PRP
                                    VP -> VBP NP
                                    VP -> VBP VP
                                    VP -> VBG NP
                                    VP -> TO VP
                                    VP -> VB
                                    VP -> VB NP
                                    NN -> "show" | "book"
                                    PRP -> "I"
                                    VBP -> "am"
                                    VBG -> "watching"
                                    VB -> "show"
                                    DT -> "a" | "the"
                                    MD -> "will"  """)

grammar3.productions()

[S -> NP VP,
 S -> VP,
 NP -> DT NN,
 NP -> DT JJ NN,
 NP -> PRP,
 VP -> VBP NP,
 VP -> VBP VP,
 VP -> VBG NP,
 VP -> TO VP,
 VP -> VB,
 VP -> VB NP,
 NN -> 'show',
 NN -> 'book',
 PRP -> 'I',
 VBP -> 'am',
 VBG -> 'watching',
 VB -> 'show',
 DT -> 'a',
 DT -> 'the',
 MD -> 'will']

In [509]:
x = Bottom_up_Parsing('am', Nonterminal('VP'), grammar3)
print(x)

[[VBP -> 'am', VP -> VBP NP], [VBP -> 'am', VP -> VBP VP]]


In [512]:
val = left_corner_parser(grammar3, 'I am watching a show')
print(val)

[NN -> 'show']
[DT -> 'a', NP -> DT NN]
[VBG -> 'watching', VP -> VBG NP]
[VBP -> 'am', VP -> VBP VP]
[PRP -> 'I', NP -> PRP, S -> NP VP]
True
